In [ ]:
!pip install --quiet transformers datasets accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00


In [ ]:
import os
import typing as t

import torch
from torch.utils.data import Dataset
from tokenizers import ByteLevelBPETokenizer
from transformers import GPT2Config, GPT2TokenizerFast, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer

In [ ]:
!rm -rf my_tokenizer
!rm -rf saltan_model

In [ ]:
class MyDataset(Dataset):

  def __init__(
      self,
      file_path: str,
      tokenizer: GPT2TokenizerFast,
      context_length: int,
  ) -> None:

    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
      all_text = f.read()

    self.file_path = file_path
    self.tokenizer = tokenizer
    self.context_length = context_length
    self.n_samples = len(all_text) - context_length
    self.all_text = all_text

  def __getitem__(self, index: int) -> t.Dict[str, torch.Tensor]:

    text_data = self.all_text[index: index + self.context_length]

    encoded = self.tokenizer(
        text_data,
        padding='max_length',
        truncation=True,
        max_length=self.context_length,
        return_tensors='pt'
    )

    return encoded


  def __len__(self) -> int:
    return self.n_samples

In [ ]:
vocab_size = 500
context_length = 128

gpt2_config = GPT2Config(
    vocab_size=vocab_size + 1,
    n_positions=context_length,
    n_embd=256,
    n_layer=4,
    n_head=2
)

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(
    files=['data.txt'],
    vocab_size=vocab_size,
    min_frequency=2,
    show_progress=True
)

if not os.path.exists('my_tokenizer'):
  os.mkdir('my_tokenizer')

tokenizer.save_model('my_tokenizer')

gpt_model = GPT2LMHeadModel(gpt2_config)
gpt_tokenizer = GPT2TokenizerFast.from_pretrained('./my_tokenizer')
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

dataset = MyDataset(
    tokenizer=gpt_tokenizer,
    file_path="./data.txt",
    context_length=context_length,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=gpt_tokenizer, mlm=False)

In [ ]:
training_args = TrainingArguments(
    output_dir="./saltan_model",
    evaluation_strategy='epoch',
    learning_rate=2e-4,
    weight_decay=0.01,
    num_train_epochs=10,
    prediction_loss_only=True,
    overwrite_output_dir=True,
    logging_steps=100
)

trainer = Trainer(
    model=gpt_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=dataset
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.369300,0.238741
2,0.240600,0.167502
3,0.196000,0.144610
4,0.172700,0.135228
5,0.157800,0.119571
6,0.143900,0.115700
7,0.133400,0.107934
8,0.126900,0.099859


In [ ]:
gpt_model = gpt_model.cpu()

In [ ]:
model_inputs = gpt_tokenizer('Белка', return_tensors='pt')
greedy_output = gpt_model.generate(**model_inputs, max_new_tokens=100)
print("Output:\n" + 100 * '-')
print(gpt_tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
Белка там живет ручная,
Да затейница какая!
Белка песенки поет,
Да орешки всё грызет,
А орешки не простые,
Всё скорлупки золоты-тос удь»
Ченя у удиве удь удь удь 
